<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Capacitated Facility Location - Problem #B



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents


- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)
- [Lowest Production Heuristic](#lowProd)
- [Greatest Production Heuristic](#maxProd)


<br>
<br>


<a id='prob'></a>
## Problem #B


<br>
Facilities <br>
$|I| = 100$ 
<br>
<br>
Clients <br>
$|J| = 1.000$ 
<br>
<br>
Capacity <br>
$u = 8.000$ 



<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        #capacity.append(file_list[i+shift])
        capacity.append(8000)
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcapa'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
facilities = list(range(num_facilities))
customers = list(range(num_customers))

capacity, fixed_cost = getFacilities_STRCapacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))

shipping_demand={}
for k, v in shipping_cost.items():
    shipping_demand[k] = v * demand[k[0]]

#setup cost
setup_cost = fixed_cost
#demand of customer
dc = demand
#max production
maxp= capacity   

In [2]:
#print(num_facilities)
#print(num_customers)
#print(capacity)
#print(fixed_cost)
#print(cartesian_prod)
#print(demand)
#print(alloc_cost)
#print(shipping_cost)
#print(shipping_demand)


In [29]:


start = time.time()
# MIP  model formulation
m = gp.Model('CFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')
m.addConstrs((gp.quicksum(dc[c]*y[(c,f)] for c in range(num_customers)) <= maxp[f]*x[f] for f in range(num_facilities)), name='Capacity')

##### Objective Function #####
#m.setObjective(x.prod(setup_cost)+y.prod(shipping_demand), GRB.MINIMIZE)
m.setObjective(x.prod(setup_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Using license file /Users/danielcinalli/gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-31
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 101100 rows, 100100 columns and 400100 nonzeros
Model fingerprint: 0x63f0d508
Variable types: 100000 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [5e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.095927e+08
Presolve time: 2.64s
Presolved: 101100 rows, 100100 columns, 400100 nonzeros
Variable types: 100000 continuous, 100 integer (100 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    9202    1.3658080e+07   3.044984e+04   0.000000e+00      5s
   18666    1

In [31]:

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")



Build a warehouse at location 16.

Build a warehouse at location 30.

Build a warehouse at location 33.

Build a warehouse at location 69.

Build a warehouse at location 70.

Build a warehouse at location 79.

Build a warehouse at location 89.


In [2]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

#m.write('UFLP_02_Simplex.lp')

NameError: name 'x' is not defined

In [3]:
from itertools import product
from math import sqrt
from copy import copy
import time
import random

# Randomly choose a Facility
def ChooseFac_rand(item):
    return random.randint(0, len(item)-1)


# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_shipping(cp_shipping_cost, client, cp_facilities, facilities):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif aux[aux_key] > cp_shipping_cost.get((client, facilities.index(j))):
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))


    return aux_key



# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_shipping(cp_shipping_cost, client, cp_facilities, facilities):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif not aux[aux_key] > cp_shipping_cost.get((client, facilities.index(j))):
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_shipping_fcost(cp_shipping_cost, client, cp_facilities, facilities, setup_cost):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        dedo = facilities.index(j)
        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif aux[aux_key] > cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]:
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_shipping_fcost(cp_shipping_cost, client, cp_facilities, facilities, setup_cost):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        dedo = facilities.index(j)
        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif not aux[aux_key] > cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]:
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_production(cp_shipping_cost, client, cp_facilities, facilities, cp_maxp):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        dedo = facilities.index(j)
        if cp_maxp[dedo] != 0:
            if not aux:
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))
                # print(aux)
                # print(aux_key)
            #if this FAC is cheaper for this client, replace the item
            elif aux[aux_key] > cp_maxp[dedo]: #cp_shipping_cost.get((client, facilities.index(j))):
                # print("....")
                # print (i[0],j[0])
                # print(aux[aux_key])
                # print(shipping_cost.get((i[0],j[0])))
                # print ("  ")
                aux.pop(aux_key)
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))


    return aux_key


# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_production(cp_shipping_cost, client, cp_facilities, facilities, cp_maxp):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        dedo = facilities.index(j)
        if cp_maxp[dedo] != 0:
            if not aux:
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))
                # print(aux)
                # print(aux_key)
            #if this FAC is cheaper for this client, replace the item
            elif not aux[aux_key] > cp_maxp[dedo]: #cp_shipping_cost.get((client, facilities.index(j))):
                # print("....")
                # print (i[0],j[0])
                # print(aux[aux_key])
                # print(shipping_cost.get((i[0],j[0])))
                # print ("  ")
                aux.pop(aux_key)
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))


    return aux_key

# Calculate the Total Cost
def CalculateCost(customers, cp_facilities, cp_setup_cost, cost_per_mile, result):
    total_dist = 0
    total_setup = 0
    for client, fac, prods in result:
        #shipping cost of demand
        #total_dist += prods * cost_per_mile * compute_distance(customers[client], fac)
        total_dist += cost_per_mile * compute_distance(customers[client], fac)

        #cost of FAC setup
        if(fac in cp_facilities):
            dedo = cp_facilities.index(fac)
            total_setup += cp_setup_cost[dedo]
            #remove FAC from the list
            del cp_setup_cost[dedo]
            del cp_facilities[dedo]


    return total_dist+total_setup


def CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile, result, dc):
    total_dist = 0
    total_setup = 0
    my_facilities = copy(cp_facilities)

    for client, fac, prods in result:
        #shipping cost of demand
        #total_dist += prods * cost_per_mile * compute_distance(customers[client], fac)
        prod_perc = float(prods)/dc[client]
        total_dist += prod_perc*cost_per_mile * compute_distance(customers[client], fac)

        #cost of FAC setup
        if(fac in cp_facilities):
            dedo = cp_facilities.index(fac)
            total_setup += cp_setup_cost[dedo]
            #remove FAC from the list
            del cp_setup_cost[dedo]
            del cp_facilities[dedo]


    return total_dist+total_setup

# Calculate the Total Cost
def CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost, result, dc):
    total_dist = 0
    total_setup = 0
    my_facilities = copy(cp_facilities)

    for client, fac, prods in result:
        #shipping cost of demand
        #total_dist += prods * cost_per_mile * compute_distance(customers[client], fac)
        prod_perc = float(prods)/dc[client]
        total_dist += prod_perc*shipping_cost[(client, fac)]

        #cost of FAC setup
        if(fac in cp_facilities):
            dedo = cp_facilities.index(fac)
            total_setup += cp_setup_cost[dedo]
            #remove FAC from the list
            del cp_setup_cost[dedo]
            del cp_facilities[dedo]


    return total_dist+total_setup

def PrintResult(result, facs):

    #length Facs
    lenFacs = len(facs)

    #print clients and facs disposition
    for i in result:
        #print("Client " + str(i[0] + 1) + "receives part of its demand  from Warehouse" + str(i[1] + 1) + ".")
        #print("Client " + str(i[0]+1) + " receives part of its demand  from Warehouse %s." % (i[1],))

        if (i[1] in facs):
            dedo= facs.index(i[1])
            del facs[dedo]

    lenFacs = lenFacs - len(facs)
    print("Number of Warehouses: " + str(lenFacs))


def PrintResult_CFLP(result, facs):

    #length Facs
    lenFacs = len(facs)

    #print clients and facs disposition
    for i in result:
        #print("Client " + str(i[0] + 1) + "receives part of its demand  from Warehouse" + str(i[1] + 1) + ".")
        #print("Client " + str(i[0]+1) + " receives part of its demand  from Warehouse %s." % (i[1]+1,))

        if (i[1] in facs):
            dedo= facs.index(i[1])
            del facs[dedo]


    lenFacs = lenFacs - len(facs)
    print("Number of Warehouses: " + str(lenFacs))



# Euclidean distance between a facility and customer sites
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)



<br>
<br>

<a id='random'></a>
### Random Heuristic 





In [4]:

result = []
#i = 0

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)

##########
# Random #
##########

# choose the Facility for each customer
for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_rand(cp_facilities)

        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[fac]<=0): #facility produces more than needed
            cp_maxp[fac] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, cp_facilities[fac], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[fac]
            # append the result: Client | FAC | total of products
            result.append((idx, cp_facilities[fac], cp_maxp[fac]))
            cp_maxp[fac] = 0

            #remove facility and its production item
            del cp_facilities[fac]
            del cp_maxp[fac]


# calculate the setup_cost
cp_facilities = copy(facilities)
cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)

208936175.27787
TIME IS:  0.007322788238525391
Number of Warehouses: 100


<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [5]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_shipping(cp_shipping_cost, idx, cp_facilities, facilities)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)

181198878.89283
TIME IS:  0.2356729507446289
Number of Warehouses: 99


<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [6]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_shipping(cp_shipping_cost, idx, cp_facilities, facilities)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)


114409197.6437611
TIME IS:  0.26480913162231445
Number of Warehouses: 32


<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [21]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_shipping_fcost(cp_shipping_cost, idx, cp_facilities, facilities, setup_cost)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)

36089807.427437544
TIME IS:  1.6648921966552734
Number of Warehouses: 10


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [22]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_shipping_fcost(cp_shipping_cost, idx, cp_facilities, facilities, setup_cost)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)



59535242.73111988
TIME IS:  1.5023701190948486
Number of Warehouses: 9


<a id='lowProd'></a>
### Lowest Production Heuristic



In [24]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_production(cp_shipping_cost, idx, cp_facilities, facilities, cp_maxp)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)


45730126.2059792
TIME IS:  0.6215469837188721
Number of Warehouses: 7


<a id='maxProd'></a>
### Greatest Production Heuristic



In [25]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx in customers:

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_production(cp_shipping_cost, idx, cp_facilities, facilities, cp_maxp)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP_OR(customers, cp_facilities, cp_setup_cost, shipping_cost,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult_CFLP(result, cp_facilities)

210037408.03646004
TIME IS:  0.709679126739502
Number of Warehouses: 100
